# Binance Trading bot

Notes:
1. Order according to moving average and volatility.
1. The rolling window is initiliazed using historical prices. 
1. Binance does not allow historical klines shorter than 1 minute. But we can trade as frequent as we want.
1. Some terminology. *Base*: means the crypto we are trading. *Quote*: Is the stablecoin we use (i.e., crypto pegged to US$).
1. Add a *credentials.json* with your api keys in the same folder as the notebook.
1. So far the script has only executed sell orders, because I kill it everyday when I go to work it never bought crypto.
1. Api Documentation: https://python-binance.readthedocs.io/en/latest/index.html

In [ ]:
%matplotlib notebook

import json
import time
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

from binance.client import Client
from binance.enums import *
from binance.helpers import round_step_size

## Connect to server

In [ ]:
creds = json.load(open('credentials.json'))

client = Client(creds['api_key'], creds['api_secret'])
msg = 'Problem connecting' if client.get_system_status()['status'] else 'Server is up'
print(msg)

## Parameters

In [ ]:
base = 'BTC'
quote = 'BUSD'
pair = base + quote

interval = 60 # check every X seconds
windowSize = 300 # rolling window length

diff = 3 # how many stds away from the avg you execute an order

## Functions

In [ ]:
# Gets current price of base denominated in quote
def getPrice():
    return float(client.get_symbol_ticker(symbol = pair)['price'])

# Gets amount of base and quote assets that are free to trade (i.e., not held by an order)
def getAssets():
    assets = {}
    wallet = client.get_account()['balances']
    for a in wallet:
        if a['asset'] == base:
            assets[base] = float(a['free'])
        if a['asset'] == quote:
            assets[quote] = float(a['free'])
    return assets   

# Gets current balance in quote
def getBalance():
    baseVal = getPrice() * getAssets()[base]
    quoteVal = getAssets()[quote]
    return baseVal + quoteVal

# Fills up the rolling window with historical prices
def getHistory():
    candles = client.get_klines(symbol = pair, interval = '1m') #candle length is 1 minute
    return [float(c[4]) for c in candles[-windowSize:]] # c[4] returns the candle's close value

# DANGER ZONE
# Executes a market order
# Use create_test_order(.) to test. If all goes well it returns {}
def makeOrder(s):
    if s == 'BUY':
        q = getAssets()[quote]
        return client.create_test_order(symbol = pair, side = s, type = 'MARKET', quoteOrderQty = q)
    q = str(getAssets()[base]) 
    q = float(q[:-1])
    return client.create_test_order(symbol = pair, side = s, type = 'MARKET', quantity = q)
    
def plot():
    for i, s in enumerate(stats):
        ax[i].plot(stats[s])
        ax[i].title.set_text(s)
    fig.tight_layout()
    fig.canvas.draw()      

## Trading loop

In [ ]:
stats = defaultdict(list)
order = 'SELL' # the first order it will execute is to sell the base
window = getHistory()

fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (10, 6))
while True:  

    window.append(getPrice())
    window.pop(0)
    
    stats['avg'].append(np.average(window))
    stats['std'].append(np.std(window))
    
    # buy crypto
    if window[-1] <= stats['avg'][-1] - diff * stats['std'][-1] and order == 'BUY':
        print(makeOrder('BUY'))
        order = 'SELL'
        
    # sell crypto
    elif window[-1] >= stats['avg'][-1] + diff * stats['std'][-1] and order == 'SELL':
        print(makeOrder('SELL'))
        order = 'BUY'

    stats['value'].append(window[-1])
    stats['balance'].append(getBalance())

    plot()
    time.sleep(interval)